In [29]:
import cv2
import tensorflow as tf
import numpy as np

# Load the Keras model from the .h5 file
model_path = 'chris11.h5'
model = tf.keras.models.load_model(model_path)

In [4]:
print(tf.__version__)

2.9.1


In [1]:
import cv2
import tensorflow as tf
import numpy as np

# Load the Keras model from the .h5 file
model_path = 'chris11.h5'
model = tf.keras.models.load_model(model_path)

# Load an image for face recognition
image_path = 'chr.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Preprocess the input image
input_shape = model.layers[0].input_shape[0][1:3]
input_image = cv2.resize(image, (input_shape[0], input_shape[1]))
input_image = np.expand_dims(input_image, axis=0)
input_image = (input_image.astype(np.float32) / 127.5) - 1.0

# Run inference
output_data = model.predict(input_image)

# Post-process the output for binary classification
confidence = output_data[0][0]  # Assuming your model output is a single value (0 or 1)

# Define a threshold for classification (e.g., 0.5)
threshold = 0.5

# Check if the confidence is above the threshold (a face is detected)
if confidence >= threshold:
    # Get the coordinates for a bounding box around the detected face
    height, width, _ = image.shape
    top, left, bottom, right = 0, 0, height, width  # Adjust these values based on your model's output

    # Draw a bounding box around the detected face
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)

    # Write "chris" on the image
    text = "chris"
    cv2.putText(image, text, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Crop the detected face
    cropped_face = image[top:bottom, left:right]

    # Save the cropped face
    cv2.imwrite('crop.jpg', cropped_face)

# Display or save the image with the bounding box
cv2.imshow('Face Recognition', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

In [32]:
from tensorflow.keras.models import load_model
from datetime import datetime, timedelta
classifier = load_model('chris11.h5')#Earth_Video_6_Tune.h5
import cv2
import csv
import numpy as np
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
times_detected = []
#start_time = datetime.now()
minutes_detected = {}


csv_file = open('encounter.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Person', 'Elapsed Time (seconds)'])

# with open('encounter.csv', mode='w', newline='') as csvfile:
#     fieldnames = ['Person', 'Elapsed Time (seconds)']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#     # Write the header row to the CSV file
#     writer.writeheader()
def face_detector(img):
    #Convert image to grayscale
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.imread('ca.jpg')
    faces = face_classifier.detectMultiScale(img,scaleFactor=1.2, minNeighbors=5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    for (x,y,w,h) in faces:
        #cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = img[y:y+h, x:x+w]

    try:
        roi_gray = cv2.resize(roi_gray, (128, 128))
    except:
        return (x,w,y,h), np.zeros((48,48), np.uint8), img
    return (x,w,y,h), roi_gray, img

img = cv2.imread('ca.jpg')
class_labels = {0: 'Chris'}

rect, face, image = face_detector(img)
print(rect)

<>:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\user\AppData\Local\Temp\ipykernel_15568\1732069760.py:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [33]:
cap = cv2.VideoCapture("chvid.mp4")
new_width = 640
new_height = 480
try:
    while True:

        ret, frame = cap.read()
        frame = cv2.resize(frame, (new_width, new_height))
        rect, face, image = face_detector(frame)
        print("rect",rect)
        print("face",face)
        if np.sum([face]) != 0.0:
            roi = face.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            # make a prediction on the ROI, then lookup the class
            preds = classifier.predict(roi)[0]
            print(preds)
            #print(p*100)
            if preds>=0.5:
                label = class_labels[preds.argmax()]
                print(label)
                label_position = (rect[0] + int((rect[1]/2)), rect[2] + 25)
                cv2.rectangle(image,(rect[0],rect[2]),(rect[0]+rect[1],rect[2]+rect[3]),(255,255,0),2)
                cv2.putText(image, label, (rect[0],rect[2]-25) , cv2.FONT_HERSHEY_COMPLEX,2, (0,255,0), 2)
                time_elapsed = cap.get(cv2.CAP_PROP_POS_MSEC)

                elapsed_seconds = time_elapsed / 1000
        #         print("time_elap",elapsed_seconds)
        #         print("label",label)
                #writer.writerow({'Person': label, 'Elapsed Time (seconds)': elapsed_seconds})
                csv_writer.writerow([label, elapsed_seconds])
                elapsed_minutes = elapsed_seconds // 60

            # Record the minute of the video at which the person appeared
#             if elapsed_minutes not in minutes_detected:
#                 minutes_detected[elapsed_minutes] = 1
#             start_time = datetime.now()
        else:
            cv2.putText(image, "No Face Found", (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)

        cv2.imshow('All', image)
        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break
except Exception as e:
    print("Error Occurs:" , e)
#csv_file.close()      
cap.release()
cv2.destroyAllWindows()

rect (0, 0, 0, 0)
face [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
rect (0, 0, 0, 0)
face [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
rect (0, 0, 0, 0)
face [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
rect (0, 0, 0, 0)
face [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
rect (0, 0, 0, 0)
face [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
rect (0, 0, 0, 0)
face [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
rect (0, 0, 0, 0)
face [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

In [27]:
import cv2
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def face_detector(img):
    faces = face_classifier.detectMultiScale(img, scaleFactor=1.2, minNeighbors=5)
    if len(faces) == 0:
        return (0, 0, 0, 0), np.zeros((48, 48), np.uint8), img

    for (x, y, w, h) in faces:
        roi_gray = img[y:y+h, x:x+w]

    try:
        roi_gray = cv2.resize(roi_gray, (128, 128))
    except:
        return (x, w, y, h), np.zeros((48, 48), np.uint8), img
    return (x, w, y, h), roi_gray, img



(1713, 81, 3811, 81)


In [23]:
print(cv2.__version__)

4.7.0


In [3]:
import cv2

# Load the pre-trained Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open a video capture stream (0 is usually the default camera)
cap = cv2.VideoCapture("chvid.mp4")

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    if not ret:
        break
    aspect_ratio = 680 / 440
    frame = cv2.resize(frame, (680, int(680 / aspect_ratio)))
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

    # Display the frame with detected faces
    cv2.imshow('Face Detection', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


# CORRECTED

In [32]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load your deep learning model for face detection
model = load_model('chris_21.h5')

# Open a video capture stream
cap = cv2.VideoCapture("chvid.mp4")

# Confidence threshold for face detection
confidence_threshold = 0.5

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    if not ret:
        break

    aspect_ratio = 680 / 440
    frame = cv2.resize(frame, (680, int(680 / aspect_ratio)))

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection using Haar cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    print(faces)
    for (x, y, w, h) in faces:
        face_roi = frame[y:y + h, x:x + w]
        # Resize the face region to the input size expected by your model
        face_roi = cv2.resize(face_roi, (224, 224))

        # Preprocess the image for the model
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0  # Normalize to [0, 1]

        # Perform face detection using the deep learning model
        confidence = model.predict(face_roi)[0]
        print(confidence[0])
        # Check if the confidence level is above the threshold
        #if confidence[0]==0.0:
        if np.any(confidence >= confidence_threshold):
            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
            cv2.putText(frame, "CHRIS", (x, y + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            # Display "No face found" if confidence is below the threshold
            cv2.putText(frame, "No face found", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Display the frame with detected faces
    cv2.imshow('Face Detection', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


[[302  64  73  73]]
1/1 [==============================] - 1s 824ms/step
0.035856564
[[302  64  73  73]]
1/1 [==============================] - 0s 70ms/step
0.03582055
[[302  64  73  73]]
1/1 [==============================] - 0s 62ms/step
0.035819724
[[302  64  73  73]]
1/1 [==============================] - 0s 68ms/step
0.03582043
[[302  64  73  73]]
1/1 [==============================] - 0s 73ms/step
0.03582489
()
()
()
()
[[302  64  73  73]]
1/1 [==============================] - 0s 79ms/step
0.03580524
[[302  64  73  73]]
1/1 [==============================] - 0s 54ms/step
0.035801165
[[302  64  73  73]]
1/1 [==============================] - 0s 71ms/step
0.035800483
[[302  64  73  73]]
1/1 [==============================] - 0s 67ms/step
0.035815027
[[302  64  73  73]]
1/1 [==============================] - 0s 67ms/step
0.035803143
[[302  64  73  73]]
1/1 [==============================] - 0s 70ms/step
0.03580258
[[302  64  73  73]]
1/1 [==============================] - 0s 55ms/s

1/1 [==============================] - 0s 65ms/step
0.035817813
[[302  64  73  73]]
1/1 [==============================] - 0s 73ms/step
0.035829224
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
[[302  64  73  73]]
1/1 [==============================] - 0s 68ms/step
0.035827644
[[302  64  73  73]]
1/1 [==============================] - 0s 73ms/step
0.035826735
[[302  64  73  73]]
1/1 [==============================] - 0s 69ms/step
0.035826735
[[302  64  73  73]]
1/1 [==============================] - 0s 71ms/step
0.035826735
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()


In [7]:
from mtcnn import MTCNN
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
#Load your Keras model
model = load_model('Chris_4.h5')

detector = MTCNN()

img = cv2.imread('ca.jpg')

output = detector.detect_faces(img)
print(output)
for i, face_info in enumerate(output):
    
    x, y, width, height = face_info['box']
    roi = img[y:y+height, x:x+width]
    print(roi.shape)
    #Make predictions on the extracted ROI using your Keras model
    face_roi = cv2.resize(roi, (224, 224))
    face_roi = np.expand_dims(face_roi, axis=0)
    face_roi = face_roi / 255.0  # Normalize to [0, 1]

        # Perform face detection using the deep learning model
    confidence = model.predict(face_roi)[0]
    print(confidence)
    img = cv2.rectangle(img, pt1=(x, y), pt2=(x+width, y+height), color=(255, 0, 0), thickness=3)
    
    con=confidence[0]*1000
    print(con)
    if con<75:
        label = 'CHRIS'  # You can replace this with the appropriate label
        img = cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
    # Draw a rectangle and label around the detected face
    #img = cv2.rectangle(img, pt1=(x, y), pt2=(x+width, y+height), color=(255, 0, 0), thickness=3)
cv2.imshow('Face Detection', roi)
cv2.waitKey(0)
cv2.destroyAllWindows()
# Save the resulting image
cv2.imwrite('output.jpg', img)


2/2 [==============================] - 0s 39ms/step
[{'box': [1302, 1097, 377, 452], 'confidence': 0.993178129196167, 'keypoints': {'left_eye': (1404, 1259), 'right_eye': (1579, 1253), 'nose': (1476, 1310), 'mouth_left': (1413, 1438), 'mouth_right': (1556, 1436)}}, {'box': [1675, 966, 28, 38], 'confidence': 0.9512816667556763, 'keypoints': {'left_eye': (1684, 980), 'right_eye': (1696, 979), 'nose': (1689, 984), 'mouth_left': (1685, 994), 'mouth_right': (1695, 993)}}, {'box': [1713, 1224, 58, 63], 'confidence': 0.9149593114852905, 'keypoints': {'left_eye': (1736, 1244), 'right_eye': (1761, 1247), 'nose': (1750, 1263), 'mouth_left': (1734, 1271), 'mouth_right': (1757, 1275)}}, {'box': [429, 3613, 27, 31], 'confidence': 0.8850687146186829, 'keypoints': {'left_eye': (438, 3623), 'right_eye': (449, 3623), 'nose': (443, 3630), 'mouth_left': (438, 3636), 'mouth_right': (446, 3637)}}, {'box': [1715, 3638, 20, 25], 'confidence': 0.7634138464927673, 'keypoints': {'left_eye': (1722, 3647), 'right

True

# CORRECTED KERAS CODE

In [31]:
from mtcnn import MTCNN
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model

# Load your Keras model
model = load_model('Chris_51.h5')#Chris_4 is a better one right now

detector = MTCNN()

img = cv2.imread('da.jpg')

output = detector.detect_faces(img)
print(output)

for i, face_info in enumerate(output):
    confidence = face_info['confidence']
    
    if confidence >= 0.97:
        x, y, width, height = face_info['box']
        roi = img[y:y+height, x:x+width]
        print(roi.shape)

        # Make predictions on the extracted ROI using your Keras model
        face_roi = cv2.resize(roi, (75, 75))
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0  # Normalize to [0, 1]

        # Perform face recognition using the deep learning model
        conf = model.predict(face_roi)[0]
        print(conf)

        img = cv2.rectangle(img, pt1=(x, y), pt2=(x+width, y+height), color=(255, 0, 0), thickness=3)

        con = conf[0] * 100
        print(con)
        
        if con > 45:
            label = 'CHRIS'  # You can replace this with the appropriate label
            img = cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Display or save the resulting image with ROIs
cv2.imshow('Face Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
# Save the resulting image
cv2.imwrite('output11.jpg', img)


2/2 [==============================] - 0s 15ms/step
[{'box': [1328, 943, 392, 474], 'confidence': 0.9884577393531799, 'keypoints': {'left_eye': (1457, 1087), 'right_eye': (1624, 1156), 'nose': (1502, 1184), 'mouth_left': (1394, 1263), 'mouth_right': (1522, 1322)}}, {'box': [1266, 3858, 27, 32], 'confidence': 0.7377626895904541, 'keypoints': {'left_eye': (1275, 3868), 'right_eye': (1287, 3869), 'nose': (1280, 3877), 'mouth_left': (1273, 3883), 'mouth_right': (1283, 3884)}}]
(474, 392, 3)
1/1 [==============================] - 1s 1s/step
[0.01086329]
1.0863292030990124


True

# h5 to TFLITE

In [18]:
import tensorflow as tf

# Load the Keras model in .h5 format
keras_model_path = 'Chris_4.h5'
keras_model = tf.keras.models.load_model(keras_model_path)

# Define the path for the quantized TensorFlow Lite model
tflite_model_path = 'Chris_4_quantized.tflite'

# Convert the Keras model to a TensorFlow Lite model with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Quantization
# tflite_model = converter.convert()

# # Save the quantized TensorFlow Lite model to the specified path
# with open(tflite_model_path, 'wb') as f:
#     f.write(tflite_model)

# print(f"Quantized TensorFlow Lite model saved to {tflite_model_path}")


# Prediction with tflite

In [22]:
from mtcnn import MTCNN
import cv2
import numpy as np
import tensorflow as tf
#from tensorflow import tflite
#import tflite_runtime.interpreter as tflite

# Load your quantized TFLite model
tflite_model_path = 'Chris_4_quantized.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

detector = MTCNN()

img = cv2.imread('ca.jpg')

output = detector.detect_faces(img)
print(output)

for i, face_info in enumerate(output):
    confidence = face_info['confidence']
    
    if confidence >= 0.95:
        x, y, width, height = face_info['box']
        roi = img[y:y+height, x:x+width]
        print(roi.shape)

        # Preprocess the input for the TFLite model
        face_roi = cv2.resize(roi, (224, 224))
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = np.array(face_roi, dtype=np.float32)
        face_roi = face_roi / 255.0  # Normalize to [0, 1]

        # Set the input tensor
        interpreter.set_tensor(input_details[0]['index'], face_roi)

        # Run inference
        interpreter.invoke()

        # Get the output
        confidence = interpreter.get_tensor(output_details[0]['index'])
        print(confidence)

        img = cv2.rectangle(img, pt1=(x, y), pt2=(x+width, y+height), color=(255, 0, 0), thickness=3)

        con = confidence[0][0] * 1000
        print(con)
        
        if con < 75:
            label = 'CHRIS'  # You can replace this with the appropriate label
            img = cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)

# Display or save the resulting image with ROIs
# cv2.imshow('Face Detection', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# Save the resulting image
cv2.imwrite('output1.jpg', img)

2/2 [==============================] - 0s 25ms/step
[{'box': [1302, 1097, 377, 452], 'confidence': 0.993178129196167, 'keypoints': {'left_eye': (1404, 1259), 'right_eye': (1579, 1253), 'nose': (1476, 1310), 'mouth_left': (1413, 1438), 'mouth_right': (1556, 1436)}}, {'box': [1675, 966, 28, 38], 'confidence': 0.9512816667556763, 'keypoints': {'left_eye': (1684, 980), 'right_eye': (1696, 979), 'nose': (1689, 984), 'mouth_left': (1685, 994), 'mouth_right': (1695, 993)}}, {'box': [1713, 1224, 58, 63], 'confidence': 0.9149593114852905, 'keypoints': {'left_eye': (1736, 1244), 'right_eye': (1761, 1247), 'nose': (1750, 1263), 'mouth_left': (1734, 1271), 'mouth_right': (1757, 1275)}}, {'box': [429, 3613, 27, 31], 'confidence': 0.8850687146186829, 'keypoints': {'left_eye': (438, 3623), 'right_eye': (449, 3623), 'nose': (443, 3630), 'mouth_left': (438, 3636), 'mouth_right': (446, 3637)}}, {'box': [1715, 3638, 20, 25], 'confidence': 0.7634138464927673, 'keypoints': {'left_eye': (1722, 3647), 'right

True

# Prediction

In [23]:
from mtcnn import MTCNN
import cv2
import numpy as np
import tensorflow as tf
import os

# Load your quantized TFLite model
tflite_model_path = 'Chris_4_quantized.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

detector = MTCNN()

# Define the input and output directories
input_directory = 'Original/'
output_directory = 'Predict/'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Loop through the images in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):  # Assuming the images are in JPG format
        # Load the image
        img = cv2.imread(os.path.join(input_directory, filename))

        # Detect faces in the image
        output = detector.detect_faces(img)

        for i, face_info in enumerate(output):
            confidence = face_info['confidence']

            if confidence >= 0.95:
                x, y, width, height = face_info['box']
                roi = img[y:y+height, x:x+width]

                # Preprocess the input for the TFLite model
                face_roi = cv2.resize(roi, (224, 224))
                face_roi = np.expand_dims(face_roi, axis=0)
                face_roi = np.array(face_roi, dtype=np.float32)
                face_roi = face_roi / 255.0  # Normalize to [0, 1]

                # Set the input tensor
                interpreter.set_tensor(input_details[0]['index'], face_roi)

                # Run inference
                interpreter.invoke()

                # Get the output
                confidence = interpreter.get_tensor(output_details[0]['index'])

                img = cv2.rectangle(img, pt1=(x, y), pt2=(x+width, y+height), color=(255, 0, 0), thickness=3)

                con = confidence[0][0] * 1000

                if con < 75:
                    label = 'CHRIS'  # You can replace this with the appropriate label
                    img = cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)

        # Save the resulting image in the output directory
        output_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_path, img)

print("Face recognition completed and images are saved in the 'Predict' folder.")


1/1 [==============================] - 0s 294ms/step


1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 42ms/step


2/2 [==============================] - 0s 30ms/step


2/2 [==============================] - 0s 27ms/step


KeyboardInterrupt: 